## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 14
sequence_length = 6

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'<unk>': 0,
 'cây': 5,
 'kẻ': 7,
 'thì': 11,
 'chí': 4,
 '<pad>': 1,
 '<sos_topic1>': 2,
 '<sos_topic2>': 3,
 'có': 6,
 'nhớ': 8,
 'nên': 9,
 'quả': 10,
 'trồng': 12,
 'ăn': 13}

In [3]:
corpus[0] = '<sos_topic1> ' + corpus[0]
corpus[1] = '<sos_topic2> ' + corpus[1]

data_x = []
data_y = []
for vector in corpus:
    vector = vector.split()    

    for i in range(1, len(vector)):
        data_x.append(vector[:i])
        data_y.append(vector[i])

print(data_x)
print(data_y)

[['<sos_topic1>'], ['<sos_topic1>', 'ăn'], ['<sos_topic1>', 'ăn', 'quả'], ['<sos_topic1>', 'ăn', 'quả', 'nhớ'], ['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ'], ['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng'], ['<sos_topic2>'], ['<sos_topic2>', 'có'], ['<sos_topic2>', 'có', 'chí'], ['<sos_topic2>', 'có', 'chí', 'thì']]
['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', 'có', 'chí', 'thì', 'nên']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):
    print(x)
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))
    
    return x_ids, vocab[y]

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

['<sos_topic1>']
['<sos_topic1>', 'ăn']
['<sos_topic1>', 'ăn', 'quả']
['<sos_topic1>', 'ăn', 'quả', 'nhớ']
['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ']
['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng']
['<sos_topic2>']
['<sos_topic2>', 'có']
['<sos_topic2>', 'có', 'chí']
['<sos_topic2>', 'có', 'chí', 'thì']


In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 1, 1, 1, 1, 1]
13

[2, 13, 1, 1, 1, 1]
10

[2, 13, 10, 1, 1, 1]
8

[2, 13, 10, 8, 1, 1]
7

[2, 13, 10, 8, 7, 1]
12

[2, 13, 10, 8, 7, 12]
5

[3, 1, 1, 1, 1, 1]
6

[3, 6, 1, 1, 1, 1]
4

[3, 6, 4, 1, 1, 1]
11

[3, 6, 4, 11, 1, 1]
9



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([10, 6])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([10])


## Train with full data

In [8]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.recurrent = nn.RNN(4, 4, batch_first=True)
        self.linear = nn.Linear(sequence_length*4, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)
        x = nn.Flatten()(x)
        x = self.linear(x)
        return x

model = TG_Model(vocab_size, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(14, 4)
  (recurrent): RNN(4, 4, batch_first=True)
  (linear): Linear(in_features=24, out_features=14, bias=True)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [10]:
# train
for _ in range(35):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.8929567337036133
2.391228199005127
1.9780728816986084
1.6139867305755615
1.275233507156372
1.0369093418121338
0.8680337071418762
0.7290904521942139
0.6078771352767944
0.5023180842399597
0.41155776381492615
0.3321951925754547
0.263028085231781
0.2066657990217209
0.16402289271354675
0.13232974708080292
0.10795924812555313
0.08847444504499435
0.072664275765419
0.05991523712873459
0.049761101603507996
0.04174325242638588
0.035425812005996704
0.030428964644670486
0.026432689279317856
0.023166561499238014
0.02041582018136978
0.018037933856248856
0.01596108451485634
0.01415717788040638
0.01261023711413145
0.011299170553684235
0.010195476934313774
0.009267406538128853
0.00848465971648693


In [11]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=-1))

tensor([13, 10,  8,  7, 12,  5,  6,  4, 11,  9])


In [12]:
data_y_ids

tensor([13, 10,  8,  7, 12,  5,  6,  4, 11,  9])

## Inference

In [13]:
promt = '<sos_topic2>'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[3, 1, 1, 1, 1, 1]


In [14]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    next_id = torch.argmax(outputs, axis=-1)

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[3, 6, 1, 1, 1, 1]
[3, 6, 4, 1, 1, 1]
[3, 6, 4, 11, 1, 1]
[3, 6, 4, 11, 9, 1]
[3, 6, 4, 11, 9, 9]


In [15]:
{'<unk>': 0,
 'cây': 5,
 'kẻ': 7,
 'thì': 11,
 'chí': 4,
 '<pad>': 1,
 '<sos_topic1>': 2,
 '<sos_topic2>': 3,
 'có': 6,
 'nhớ': 8,
 'nên': 9,
 'quả': 10,
 'trồng': 12,
 'ăn': 13}

{'<unk>': 0,
 'cây': 5,
 'kẻ': 7,
 'thì': 11,
 'chí': 4,
 '<pad>': 1,
 '<sos_topic1>': 2,
 '<sos_topic2>': 3,
 'có': 6,
 'nhớ': 8,
 'nên': 9,
 'quả': 10,
 'trồng': 12,
 'ăn': 13}